In [1]:
import time
import os
import mne
import pprint
import numpy as np
import pandas as pd
from matplotlib.pyplot import plot

import brainflow
from brainflow.board_shim import BoardIds, BrainFlowInputParams, BoardShim
from brainflow.data_filter import DataFilter

In [2]:
mi_class = "left"  # motor imagey class for this session

# main directory
main_dir = os.getcwd()
# eeg data directory
eeg_dir = os.path.join(main_dir, "raw_eeg_data", str(mi_class))
print("Saving data in: " + str(eeg_dir))

# OpenBCI Cyton board
cyton_board_ID = 0
#print("\n" + str(BoardIds(cyton_board_ID)))  

# serial connection
serial_port = 'COM3'
params = BrainFlowInputParams()
params.serial_port = serial_port

# create board
board = BoardShim(board_id=cyton_board_ID, input_params=params)

# board attributes
board_name = board.get_device_name(board_id=board.board_id)
channel_names = board.get_eeg_names(board_id=board.board_id)
samp_freq = board.get_sampling_rate(board_id=board.board_id)  # 250 Hz
eeg_channel_idx = board.get_eeg_channels(board_id=board.board_id)  # EEG channel indices (1-8)

# prepare the board for data stream
board.prepare_session()
print("\nSession ready.")
print("--------------")
# start data stream
board.start_stream()
print("\nStreaming data...")

# number of seconds to record
time.sleep(10)

# get all data
data = board.get_board_data()  
# stop the data stream
board.stop_stream()
board.release_session()
print("Session ended.")

Saving data in: c:\Users\Zack\OneDrive\Drexel\2021 - 2022\BMES 725\Final Project\BMES725-BCI-Motor-Imagery\raw_eeg_data\left

Session ready.
--------------

Streaming data...
Session ended.


In [3]:
data_shape = np.shape(data)
num_rows = data_shape[0]
num_cols = data_shape[1]

# add MI class column
class_col = np.array([[mi_class for x in range(num_cols)]])
data = np.vstack((data, class_col))

In [10]:
timestamp_channel = board.get_timestamp_channel(board_id=board.board_id)
initial_timestamp = data[timestamp_channel, 0]

# save recorded eeg data
filename = str(initial_timestamp) + '.txt'
filepath = os.path.join(eeg_dir, filename)
os.chdir(eeg_dir)
#DataFilter.write_file(data, filename, 'w')
np.savetxt(filename, X=data, delimiter=",", fmt="%s")
os.chdir(main_dir)

In [11]:
all_raw_data = np.loadtxt(filepath, delimiter=',')

ValueError: could not convert string to float: 'left'